# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
filepath="../output_data/cities.csv"
cities_df=pd.read_csv(filepath)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vostok,46.49,135.88,30.15,92,78,5.77,RU,1602595218
1,Palmer,42.16,-72.33,52.00,100,90,8.05,US,1602595343
2,Caravelas,-17.71,-39.25,77.52,85,55,9.13,BR,1602595364
3,Chālūs,36.66,51.42,71.60,56,0,2.24,IR,1602595085
4,Lebu,-37.62,-73.65,53.82,80,0,18.68,CL,1602595364


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [130]:
locations=cities_df[['Lat','Lng']]
weights=cities_df['Humidity']
# mid_map is the Lat and Lng coordinates so I could center the map on the screen.
mid_map=(0,15)
fig=gmaps.figure(center=mid_map,zoom_level=1)
heat_layer=gmaps.heatmap_layer(locations,weights=weights)
heat_layer.max_intensity=100
heat_layer.point_radius=5
heat_layer.dissipation=False
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
hitemp=95
lwtemp=85
humidity=85
cloudiness=25
windspeed=5
my_spots_df=cities_df.loc[
                    (cities_df['Max Temp']>lwtemp) & (cities_df['Max Temp']<hitemp) &
                    (cities_df['Humidity']<humidity) &
                    (cities_df['Cloudiness']<cloudiness) & 
                    (cities_df['Wind Speed']<windspeed) 
                    ,:]
my_spots_df              

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
178,Al Bardīyah,31.76,25.09,88.45,31,0,2.64,LY,1602595381
268,Guánica,17.97,-66.91,86.00,66,0,1.01,PR,1602595389
279,Sikasso,10.92,-7.00,90.45,50,0,3.49,ML,1602595113
296,Demba,-5.51,22.27,92.50,36,14,3.60,CD,1602595391
373,Mineiros,-17.57,-52.55,88.50,41,0,1.77,BR,1602595147
411,Khanpur,28.65,70.66,85.95,13,0,3.22,PK,1602595401
448,Tsabong,-26.02,22.40,85.95,14,0,1.70,BW,1602595404
466,Garoowe,8.41,48.48,90.10,35,20,4.70,SO,1602595406
536,Birao,10.28,22.79,92.88,44,1,2.13,CF,1602595414


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [131]:
hotel_df=my_spots_df.copy()
hotel_df['Hotel Name']=''

params={
        "query":'hotel',
        "radius":5000,
        "type":"lodging",
        "language":"en",
        "key":g_key
}

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f"{lat},{lng}"
    hotel_names=requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_names["results"][0]["name"]
    except:
        hotel_df.loc[index,"Hotel Name"]="No Hotel"
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
178,Al Bardīyah,31.76,25.09,88.45,31,0,2.64,LY,1602595381,بورتو بردية
268,Guánica,17.97,-66.91,86.00,66,0,1.01,PR,1602595389,Copamarina Beach Resort & Spa
279,Sikasso,10.92,-7.00,90.45,50,0,3.49,ML,1602595113,No Hotel
296,Demba,-5.51,22.27,92.50,36,14,3.60,CD,1602595391,No Hotel
373,Mineiros,-17.57,-52.55,88.50,41,0,1.77,BR,1602595147,Pilões Palace Hotel
411,Khanpur,28.65,70.66,85.95,13,0,3.22,PK,1602595401,Khan Hotel
448,Tsabong,-26.02,22.40,85.95,14,0,1.70,BW,1602595404,Springbok Motel
466,Garoowe,8.41,48.48,90.10,35,20,4.70,SO,1602595406,Curubo Hotel
536,Birao,10.28,22.79,92.88,44,1,2.13,CF,1602595414,No Hotel


In [133]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [136]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))